In [9]:
import json

import fire
import typer
from datasets import load_dataset

import spacy
from pathlib import Path
from spacy.tokens import DocBin, Span
from typing import List

In [86]:
count = 10
# count = float("inf")
def retokenize(tokens: List) -> str:
    return " ".join(tokens)


span_key = "sc"
docs = []
lang = "en"
nlp = spacy.blank(lang)


def hf_to_span(data, tags_key: List):
    for i, element in enumerate(data):
        if i > count:
            break
        sentence = retokenize(element[TOKENS])
        doc = nlp(sentence)
        ner_tags = element["ner_tags"]
        ner_tags = [tags_key[k] for k in ner_tags]
        char_spans, char_len = [], 0
        substring = ""
        j, prev_begin = 0, False
        for j, tag in enumerate(ner_tags):
            token = element["tokens"][j]
            if "B-" in tag:
                if prev_begin:
                    char_spans.append(
                        {
                            "start": start,
                            "end": char_len,
                            "label": label,
                        }
                    )
                    substring = ""
                substring += token
                label = tag[2:]
                start = char_len
                prev_begin = True
            if "I-" in tag:
                substring += " "
                substring += token
                prev_begin = False
            if "O" == tag and len(substring) > 0:
                char_spans.append({"start": start, "end": char_len, "label": label})
                substring = ""
                prev_begin = False
            char_len += len(token)
            char_len += 1  # for the space

        if len(substring) > 0:  # jugaad for when the last tag is "I-"
            char_spans.append(
                {
                    "start": start,
                    "end": char_len,
                    "label": label,
                }
            )
        for span in char_spans:
            start, end, label = span["start"], span["end"], span["label"]
            new_ent = doc.char_span(start, end - 1, label=label)
            doc.set_ents([new_ent], default="unmodified")
        docs.append(doc)


def make_spans(dataset, output_path: Path, splits=["train", "dev", "test"]):

    tags_key = dataset[splits[0]].features["ner_tags"].feature.names
    for split in splits:
        print(f"------------{split.upper()}------------------")
        hf_to_span(dataset[split], tags_key=tags_key)


def download(output_path: Path) -> None:
    dataset = load_dataset("conll2003")
    data_dir = Path("./assets")
    output_path = data_dir / output_path
    splits = list(dataset.keys())
    make_spans(dataset, output_path, splits)


loc = Path(".")
download(".")
# docbin = DocBin().from_disk(loc)
# docs = list(docbin.get_docs(nlp.vocab))  # this is the line that I want to change
for doc in docs:
    doc.spans[span_key] = list(doc.ents)
    print([(ent, ent.label_) for ent in doc.ents])
# DocBin(docs=docs).to_disk(loc)

Reusing dataset conll2003 (C:\Users\nirantk\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


------------TRAIN------------------
------------VALIDATION------------------
------------TEST------------------
[(EU, 'ORG'), (German, 'MISC'), (British, 'MISC')]
[(Peter Blackburn, 'PER')]
[(BRUSSELS, 'LOC')]
[(European Commission, 'ORG'), (German, 'MISC'), (British, 'MISC')]
[(Germany, 'LOC'), (European Union, 'ORG'), (Werner Zwingmann, 'PER'), (Britain, 'LOC')]
[(Commission, 'ORG'), (Nikolaus van der Pas, 'PER')]
[(European Union, 'ORG')]
[(EU, 'ORG'), (Franz Fischler, 'PER')]
[(Fischler, 'PER'), (EU-wide, 'MISC'), (Britain, 'LOC'), (France, 'LOC'), (Bovine Spongiform Encephalopathy, 'MISC'), (BSE, 'MISC')]
[(Fischler, 'PER'), (EU, 'ORG')]
[(Spanish, 'MISC'), (Loyola de Palacio, 'PER'), (Fischler, 'PER'), (EU, 'ORG')]
[(LEICESTERSHIRE, 'ORG')]
[(LONDON, 'LOC')]
[(West Indian, 'MISC'), (Phil Simmons, 'PER'), (Leicestershire, 'ORG'), (Somerset, 'ORG')]
[(Essex, 'ORG'), (Derbyshire, 'ORG'), (Surrey, 'ORG'), (Kent, 'ORG'), (Nottinghamshire, 'ORG')]
[(Somerset, 'ORG'), (Grace Road, 'LOC'